In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
testDays = 7  # 测试天数
testStartDate = pd.to_datetime('2025-07-01')
testEndDate = testStartDate + pd.Timedelta(days=testDays)
# print(testDays, testStartDate, testEndDate)

# === 加载数据 ===
df = pd.read_csv('spy_minute.csv')
df.columns = [col.strip().lower() for col in df.columns]  # 标准化列名

df['date'] = pd.to_datetime(df['date'])
df['datetime'] = df['date'] + pd.to_timedelta(df['minute'], unit='m')
df.set_index('datetime', inplace=True)
df = df.sort_index()

df = df[df['date'] >= testStartDate]  # 只保留从指定日期开始的数据
df = df[df['date'] < testEndDate]  # 只保留到指定日期结束的数据

# === 计算滚动高低 ===
df['rolling_high'] = df['high'].shift(1).rolling(5).max()
df['rolling_low'] = df['low'].shift(1).rolling(5).min()
df['date'] = df['date'].astype(str)
# print(df.head(10))


In [ ]:
def _plot_result(equity_df, daily_summary):
    fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=False)
    equity_df['equity'].plot(ax=axes[0], title='Equity Curve')
    axes[0].set_ylabel('Equity ($)')
    daily_summary['daily_pnl'].plot(kind='bar', ax=axes[1], title='Daily PnL', color='orange')
    axes[1].set_ylabel('PnL ($)')
    plt.tight_layout()
    plt.show()

In [ ]:
def run_single_day(day, date, cash, window):
    position = 0
    entry = None
    trades = []
    equity_curve = []

    day['rolling_high'] = day['high'].shift(1).rolling(window).max()
    day['rolling_low'] = day['low'].shift(1).rolling(window).min()

    for i in range(window, len(day)):
        row = day.iloc[i]
        now = row.name
        price = row['close']
        highN = row['rolling_high']
        lowN = row['rolling_low']

        if i == len(day) - 1 and position != 0:
            pnl = (price - entry['price']) * position
            cash += pnl
            trades.append({
                'date': date,
                'entry_time': entry['time'],
                'exit_time': now,
                'entry_price': entry['price'],
                'exit_price': price,
                'position': position,
                'pnl': pnl
            })
            position = 0
            entry = None

        if price > highN:
            if position == -1:
                pnl = (price - entry['price']) * position
                cash += pnl
                trades.append({
                    'date': date,
                    'entry_time': entry['time'],
                    'exit_time': now,
                    'entry_price': entry['price'],
                    'exit_price': price,
                    'position': position,
                    'pnl': pnl
                })
                position = 0
            if position == 0:
                position = 1
                entry = {'time': now, 'price': price}

        elif price < lowN:
            if position == 1:
                pnl = (price - entry['price']) * position
                cash += pnl
                trades.append({
                    'date': date,
                    'entry_time': entry['time'],
                    'exit_time': now,
                    'entry_price': entry['price'],
                    'exit_price': price,
                    'position': position,
                    'pnl': pnl
                })
                position = 0
            if position == 0:
                position = -1
                entry = {'time': now, 'price': price}

        unrealized = (price - entry['price']) * position if position != 0 else 0
        equity = cash + unrealized
        equity_curve.append({'time': now, 'equity': equity})

    return trades, equity_curve, cash

In [ ]:
def run_intraday_highlow_strategy(df, startDate, days=7, initial_cash=100_000, window=5, plot=True):

    df['datetime'] = pd.to_datetime(df['date']) + pd.to_timedelta(df['minute'], unit='m')
    df = df.sort_values('datetime')
    df.set_index('datetime', inplace=True)
    df['date'] = df.index.date

    grouped = df.groupby('date')
    cash = initial_cash
    equity = cash

    all_trades = []
    equity_curve = []

    count = 0
    for date, day in grouped:
        if startDate and date < startDate:
            continue
        
        count += 1
        if count > days:
            break
        trades, day_equity_curve, cash = run_single_day(day.copy(), date, cash, window)
        all_trades.extend(trades)
        equity_curve.extend(day_equity_curve)
    
    if len(all_trades) == 0:
        print("No trades executed in the given period.")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    trades_df = pd.DataFrame(all_trades)
    equity_df = pd.DataFrame(equity_curve).set_index('time')

    daily_summary = trades_df.groupby('date').agg(
        daily_pnl=('pnl', 'sum'),
        trades=('pnl', 'count'),
        win_rate=('pnl', lambda x: (x > 0).mean())
    )

    if plot:
        _plot_result(equity_df, daily_summary)

    return trades_df, equity_df, daily_summary

In [ ]:
# === 加载数据 ===
df = pd.read_csv('spy_minute.csv')
df.columns = [col.strip().lower() for col in df.columns]  # 标准化列名
startDate = pd.to_datetime('2025-01-01').date()  # Convert to date object
days = 150
trades_df, equity_df, daily_summary = run_intraday_highlow_strategy(df, startDate=startDate, days=days, initial_cash=100_000, window=5, plot=True)
print(trades_df.head())
print(equity_df.head())
print(daily_summary.head())
print(daily_summary.tail())
